# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [2]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
#plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')
plt.imshow(grayscale(image), cmap='gray')

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)


NameError: name 'grayscale' is not defined

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (200,) * channel_count
    else:
        ignore_mask_color = 200
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=6):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
     #vertices = np.array([[(0,imshape[0]),(450, 325), (520, 325), (900,imshape[0])]], dtype=np.int32)
    one = []
    second = []
    control = 0 
    controla = 0 
    xa = 0 
    ya = 0
    xb = 0 
    yb = 0
    xy = 0 
    yy = 0
    xz = 0 
    yz = 0
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x2-x1) !=0:
                print((y2-y1)/(x2-x1))
                if ((y2-y1)/(x2-x1))<0:
                    if control == 0:
                            xa=x1
                            ya = y1
                            xb = x2
                            yb = y2
                            #cv2.line(img,(xa,ya),(xb,yb),color,10)
                            control =1 
                    else:
                        if y1>yb:
                            #cv2.line(img,(xb,yb),(x1,y1),color,10)
                            #cv2.line(img,(x1,y1),(x2,y2),color,10)
                            xb = x2
                            yb = y2
                        else:
                            if y2<ya:
                                #cv2.line(img,(x1,y1),(x2,y2),color,10)
                                #cv2.line(img,(x2,y2),(xa,ya),color,10)
                                xa = x1
                                ya = y1
                else:
                    if controla == 0:
                            xy=x1
                            yy = y1
                            xz = x2
                            yz = y2
                            #cv2.line(img,(xy,yy),(xz,yz),color,10)
                            controla =1 
                    else:
                        if y1>yz:
                            #cv2.line(img,(xz,yz),(x1,y1),color,10)
                            #cv2.line(img,(x1,y1),(x2,y2),color,10)
                            xz = x2
                            yz = y2
                        else:
                            if y2<yy:
                                #cv2.line(img,(x1,y1),(x2,y2),color,10)
                                #cv2.line(img,(x2,y2),(xy,yy),color,10)
                                xy = x1
                                yy = y1
    cv2.line(img,(xa,ya),(xb,yb),color,10)
    cv2.line(img,(xy,yy),(xz,yz),color,10)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
import os
os.listdir("test_images/")

['solidWhiteRight.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [5]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.
#plt.imshow(grayscale(image), cmap='gray')
def drawLineOnAnImage(image):
    
    image =mpimg.imread(image)
    grayImage =grayscale(image)
    imshape = image.shape

    #region_of_interest(grayImage,vertices)
    #plt.imshow(grayImage, cmap='gray')
    kernel_size = 3
    bluredImage = gaussian_blur(grayImage,kernel_size)
    low_threshold = 150
    high_threshold = 200
    vertices = np.array([[(0,imshape[0]),(460, 320), (510, 320), (900,imshape[0])]], dtype=np.int32)

    cannyImage = canny(bluredImage,low_threshold,high_threshold)
    interestedPartOfTheImage = region_of_interest(cannyImage,vertices)

    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 10 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    linedImage = hough_lines(interestedPartOfTheImage, rho, theta, threshold, min_line_len, max_line_gap)
    weightedImage = weighted_img(linedImage,image)
    plt.imshow(weightedImage,cmap='gray')
    return weightedImage

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
def process_image(image):
    print('This image is:', type(image), 'with dimensions:', image.shape)
    imagea = (image)
    
    #imshape = imagea.shape
    #vertices = np.array([[(0,imshape[0]),(600, 400), (800, 400), (1140,imshape[0])]], dtype=np.int32)
    #interestedPartOfTheImage = region_of_interest(imagea,vertices)
    
    #grayImage =grayscale(interestedPartOfTheImage)
    grayImage =grayscale(imagea)
   

    #region_of_interest(grayImage,vertices)
    #plt.imshow(grayImage, cmap='gray')
    kernel_size = 1
    bluredImage = gaussian_blur(grayImage,kernel_size)
    low_threshold = 150
    high_threshold = 200
 

    cannyImage = canny(bluredImage,low_threshold,high_threshold)
    imshape = imagea.shape
    #vertices = np.array([[(0,imshape[0]),(650, 325), (720, 325), (900,imshape[0])]], dtype=np.int32)
    vertices = np.array([[(0,imshape[0]),(450, 325), (520, 325), (900,imshape[0])]], dtype=np.int32)
   
    interestedPartOfTheImage = region_of_interest(cannyImage,vertices)
    


    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 5 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    
    #linedImage = hough_lines(cannyImage, rho, theta, threshold, min_line_len, max_line_gap)
    linedImage = hough_lines(interestedPartOfTheImage, rho, theta, threshold, min_line_len, max_line_gap)
    
    weightedImage = weighted_img(linedImage,image)
    #plt.imshow(weightedImage,cmap='gray')

        # NOTE: The output you return should be a color image (3 channel) for processing video below
        # TODO: put your pipeline here,
        # you should return the final output (image where lines are drawn on lanes)

    return weightedImage

Let's try the one with the solid white lane on the right first ...

In [8]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.603448275862
0.636363636364
0.623287671233
-0.747747747748
-0.705128205128
0.555555555556
0.677419354839
0.555555555556
0.660714285714
0.600840336134
-0.709677419355
-0.6
-0.694444444444
-0.714285714286
0.625
-0.657142857143
-0.709677419355
0.714285714286
-0.769230769231
0.65
-0.711111111111
0.6875
0.695652173913
-0.766666666667
0.621621621622
-0.615384615385
-0.631578947368
-0.785714285714
0.657142857143
-0.75
[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


  3%|▎         | 4/126 [00:00<00:03, 31.78it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.603448275862
0.636363636364
0.623287671233
-0.747747747748
-0.705128205128
0.555555555556
0.677419354839
0.555555555556
0.660714285714
0.600840336134
-0.709677419355
-0.6
-0.694444444444
-0.714285714286
0.625
-0.657142857143
-0.709677419355
0.714285714286
-0.769230769231
0.65
-0.711111111111
0.6875
0.695652173913
-0.766666666667
0.621621621622
-0.615384615385
-0.631578947368
-0.785714285714
0.657142857143
-0.75
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.604651162791
0.646288209607
-0.752066115702
-0.722222222222
0.583333333333
0.613636363636
0.583333333333
0.666666666667
-0.65
-0.714285714286
-0.6
0.5
0.677419354839
-0.703703703704
-0.708333333333
-0.735294117647
-0.764705882353
0.65
0.5625
0.590909090909
-0.615384615385
-0.740740740741
0.657894736842
-0.785714285714
-0.7
0.666666666667
-0.705882352941
0.684210526316
0.571428571429
0.609375
0.625
-0.714285714286
-0.666666666667
This ima

 10%|▉         | 12/126 [00:00<00:03, 34.19it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.6
0.624633431085
0.597222222222
0.60071942446
-0.736842105263
-0.666666666667
-0.710526315789
-0.763157894737
0.599337748344
-0.771428571429
-0.736842105263
-0.736842105263
0.625
0.571428571429
-0.636363636364
-0.75
-0.7
0.0
0.576923076923
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.628318584071
0.600649350649
-0.707317073171
0.6
0.6
-0.733333333333
0.607142857143
-0.65306122449
0.595744680851
0.614583333333
0.612903225806
-0.707317073171
0.614035087719
0.625
-0.684210526316
-0.8
-0.8
-0.714285714286
0.5625
-0.727272727273
-0.722222222222
-0.736842105263
-0.6
-0.65
0.0
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.60113960114
0.632653061224
0.61038961039
0.604166666667
0.608695652174
-0.740740740741
0.61038961039
-0.701754385965
0.595744680851
-0.733333333333
-0.684210526316
0.576923076923
0.6
-0.733333333333
-0.697674418605
0.576923076923
-0.71875
0.60869565217

...

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.644171779141
0.675409836066
-0.723076923077
-0.645569620253
0.661764705882
-0.6
0.643835616438
0.674418604651
-0.629032258065
0.685714285714
-0.615384615385
0.642857142857
0.7
-0.652173913043
0.631578947368
-0.710144927536
-0.666666666667
0.625
-0.692307692308
0.673469387755
-0.64
-0.6
0.69696969697
0.666666666667
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.672955974843
0.64375
0.674157303371
-0.669902912621
-0.69387755102
0.641975308642
0.644859813084
0.659574468085
-0.694444444444
-0.645161290323
-0.728813559322
0.666666666667
-0.6
-0.642857142857
-0.6875
-0.666666666667
-0.642857142857
-0.6
-0.6
-0.636363636364
-0.583333333333
0.68
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.673015873016
0.628865979381
0.672932330827
-0.7
-0.674074074074
0.638554216867
-0.65625
0.660714285714
-0.647058823529
-0.723076923077
-0.686274509804
-0.692307692308
-0.68
-0.666666666

 98%|█████████▊| 123/126 [00:07<00:00, 13.84it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.632352941176
0.628019323671
-0.65
0.607142857143
-0.721739130435
-0.674418604651
0.642857142857
0.659574468085
0.708333333333
0.631578947368
0.636363636364
-0.692307692308
-0.721649484536
0.626984126984
-0.625
-0.6
0.647058823529
-0.666666666667
-0.722222222222
0.652777777778
0.5
0.68
-0.625
0.5
0.5625
0.5
-0.636363636364
0.620689655172
-0.666666666667
0.571428571429
-0.6
0.666666666667
0.655737704918
-0.722222222222
0.478260869565
-0.6
-0.8125
-0.739130434783
-0.666666666667
-0.666666666667
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
0.672354948805
0.628571428571
-0.65306122449
-0.697674418605
0.643410852713
-0.690476190476
-0.744186046512
-0.583333333333
-0.769230769231
-0.673913043478
0.666666666667
-0.590909090909
0.7
-0.766666666667
0.722222222222
-0.62962962963
0.736842105263
0.727272727273
0.658536585366
0.666666666667
0.692307692308
0.652173913043
0.6875
0.727272727273
-0.696629213

 99%|█████████▉| 125/126 [00:07<00:00, 13.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.04 s, sys: 161 ms, total: 2.2 s
Wall time: 9.34 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [10]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.646226415094
-0.603658536585
0.7
0.654545454545
-0.645348837209
-0.601731601732
-0.671641791045
0.714285714286
0.649122807018
-0.708333333333
0.6
0.692307692308
0.637931034483
-0.666666666667
-0.607142857143
0.727272727273
0.684210526316
0.688888888889
-0.75
-0.666666666667
0.620689655172
0.710526315789
0.692307692308
[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


  0%|          | 3/682 [00:00<00:22, 30.00it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.646226415094
-0.603658536585
0.7
0.654545454545
-0.645348837209
-0.601731601732
-0.671641791045
0.714285714286
0.649122807018
-0.708333333333
0.6
0.692307692308
0.637931034483
-0.666666666667
-0.607142857143
0.727272727273
0.684210526316
0.688888888889
-0.75
-0.666666666667
0.620689655172
0.710526315789
0.692307692308
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.62874251497
0.647058823529
-0.636363636364
0.678899082569
0.657894736842
-0.603125
0.675
-0.609756097561
0.633802816901
-0.611111111111
0.614035087719
0.666666666667
0.666666666667
-0.603015075377
-0.6
0.673469387755
0.625
0.636363636364
-0.6
0.607142857143
-0.595238095238
0.677419354839
0.647058823529
0.647058823529
-0.623762376238
0.642857142857
-0.705882352941
0.6875
0.631578947368
-0.727272727273
-0.714285714286
-0.601973684211
-0.727272727273
0.653846153846
-0.714285714286
0.655172413793
This image is: <class 'numpy.ndarray

  2%|▏         | 11/682 [00:00<00:20, 33.27it/s]


-0.651685393258
0.62962962963
0.666666666667
0.666666666667
-0.65306122449
-0.782608695652
0.666666666667
-0.642857142857
0.655172413793
0.714285714286
0.666666666667
-0.931034482759
-0.651376146789
0.555555555556
-0.583333333333
-0.583333333333
0.722222222222
-0.571428571429
0.606060606061
-0.791666666667
0.666666666667
0.636363636364
-0.6
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.627240143369
-0.651583710407
0.676470588235
0.64
-0.631578947368
0.625
-0.8
-0.658536585366
-0.650887573964
0.648648648649
0.681818181818
0.666666666667
-0.637931034483
0.645161290323
-0.666666666667
-0.777777777778
-0.625
0.695652173913
-0.777777777778
0.625
0.7
0.714285714286
0.6875
0.7
-0.727272727273
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.627306273063
-0.668789808917
0.625
-0.7
0.697674418605
0.627906976744
-0.727272727273
0.666666666667
0.675
-0.698630136986
0.714285714286
-0.6875
0.694444444444
0.272727272727
-0.703703703704
-0.703703703

...

 99%|█████████▉| 678/682 [00:51<00:00, 12.95it/s]

-0.74358974359
-0.701298701299
-0.77358490566
-0.761904761905
-0.695652173913
0.58064516129
-0.909090909091
0.608695652174
0.6
-0.783783783784
-0.7
-0.791666666667
0.566666666667
0.616666666667
-0.740740740741
-0.846153846154
-0.683333333333
-0.9
-0.736842105263
-0.714285714286
-0.75
-0.666666666667
-0.642857142857
0.592592592593
0.583333333333
-0.647058823529
0.658536585366
0.5
-0.758620689655
-0.685714285714
-0.739130434783
-0.8
0.5
-0.705882352941
0.607142857143
-0.769230769231
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.750957854406
-0.700421940928
0.625
0.588235294118
0.588235294118
0.630434782609
-0.677966101695
0.555555555556
-0.739726027397
-0.8125
0.633333333333
-0.7
-0.707317073171
-0.688888888889
-0.764705882353
0.7
-0.745098039216
-0.677966101695
0.428571428571
0.181818181818
0.0
-0.733333333333
-0.125
0.478260869565
-0.681818181818
-0.9
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.754385964912
-0.70325203252
-0.71698

100%|█████████▉| 681/682 [00:51<00:00, 13.21it/s]

This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.753968253968
-0.700892857143
-0.736111111111
-0.712328767123
0.636363636364
0.6
0.574468085106
-0.714285714286
0.6
-0.738095238095
0.583333333333
0.615384615385
0.571428571429
-0.686274509804
0.548387096774
-0.692307692308
0.5
-0.733333333333
0.585365853659
-0.686274509804
0.571428571429
-0.75
0.583333333333
-0.0416666666667
-0.772727272727
-0.739130434783
0.588235294118
This image is: <class 'numpy.ndarray'> with dimensions: (540, 960, 3)
-0.704142011834
-0.753623188406
0.625
0.6
0.605263157895
-0.707070707071
0.615384615385
-0.752895752896
0.583333333333
0.66
0.647058823529
0.59375
0.615384615385
0.567567567568
-0.688888888889
-0.775
0.615384615385
0.529411764706
-0.739130434783
0.526315789474
0.555555555556
0.555555555556
-0.739130434783
-0.789473684211
-0.74
0.588235294118
0.625
0.0


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 11.5 s, sys: 1.13 s, total: 12.7 s
Wall time: 53.4 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))